In [1]:
from __future__ import print_function
import keras
from keras.models import Sequential, Model, load_model
from keras import backend as K

import tensorflow as tf

import os
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm

import aparent.visualization as vis

from aparent.predictor import *

import urllib
import urllib.request
import pickle
from time import sleep


Using TensorFlow backend.


In [2]:
#Load APADB Data and filter on targeted genes

#genes = ['RUNX1', 'CEBPA', 'GATA2', 'ANKRD26', 'DDX41', 'ETV6', 'PTEN', 'BRCA1', 'BRCA2', 'TP53', 'APC', 'ATM', 'PALB2', 'MSH2', 'MLH1', 'MSH6', 'PMS2', 'MUTYH']

polyadb_df = pd.read_csv('../../data/native_data/polyadb/processed_data/polyadb_processed.csv', sep=',')

#polyadb_df = polyadb_df.loc[polyadb_df['gene'].isin(genes)].reset_index(drop=True).copy()
polyadb_df = polyadb_df.loc[~polyadb_df['gene'].isna()].reset_index(drop=True).copy()
polyadb_df = polyadb_df.loc[polyadb_df['gene'] != 'na'].reset_index(drop=True).copy()

print('len(polyadb_df) = ' + str(len(polyadb_df)))


len(polyadb_df) = 190117


In [3]:
polyadb_df_minus = polyadb_df.query("strand == '-'").copy().reset_index(drop=True)
polyadb_df_plus = polyadb_df.query("strand == '+'").copy().reset_index(drop=True)

polyadb_df_minus = polyadb_df_minus.sort_values(by='pas_pos', ascending=False).copy().reset_index(drop=True)
polyadb_df_plus = polyadb_df_plus.sort_values(by='pas_pos', ascending=True).copy().reset_index(drop=True)

new_gene_id_list_plus = []
sitenum_list_plus = []
gene_id_dict = {}
for _, row in polyadb_df_plus.iterrows() :

    gene = str(row['gene'])

    if gene not in gene_id_dict :
        gene_id_dict[gene] = 0

    gene_id_dict[gene] += 1

    new_gene_id_list_plus.append(gene + "." + str(gene_id_dict[gene]))
    sitenum_list_plus.append(gene_id_dict[gene])

polyadb_df_plus['gene_id'] = new_gene_id_list_plus
polyadb_df_plus['sitenum'] = sitenum_list_plus

new_gene_id_list_minus = []
sitenum_list_minus = []
gene_id_dict = {}
for _, row in polyadb_df_minus.iterrows() :

    gene = str(row['gene'])

    if gene not in gene_id_dict :
        gene_id_dict[gene] = 0

    gene_id_dict[gene] += 1

    new_gene_id_list_minus.append(gene + "." + str(gene_id_dict[gene]))
    sitenum_list_minus.append(gene_id_dict[gene])

polyadb_df_minus['gene_id'] = new_gene_id_list_minus
polyadb_df_minus['sitenum'] = sitenum_list_minus

polyadb_df = pd.concat([polyadb_df_plus, polyadb_df_minus])

polyadb_df = polyadb_df.sort_values(by=['gene', 'sitenum'], ascending=True).reset_index(drop=True).copy()

In [4]:
polyadb_df.head()

,gene,gene_id,sitenum,num_sites,pas,seq,seq_ext,wide_seq,wide_seq_ext,rpm,total_rpm,ratio,pas_pos,cut_mode,chrom,strand,site_type
0,A1BG,A1BG.1,1,11,0,TTCTGCTAGGCTGGGCTAGTATCTTCCATGGCAAGATACTCAAACT...,TAGGGAATATAACTTTTAAAATGCGAAAGTATATTATACATATTGC...,AGTTTGGCTAGCATTTCAAAATGGATAGGGAATATAACTTTTAAAA...,GCTTTCAAAATTCCTGATTAGCTTACAATTAATTGCAAATAACTTC...,1.564234,24.390222,0.064134,58859852,58859830,chr19,-,Intron
1,A1BG,A1BG.2,2,11,-1,AGGTGCCGCTACCGCTCCTGGGTGCCCCACACCTTCGAATCGGAGC...,CGGCGAACCTCGAGCTGATCTTCGTGGGGCCCCAGCACGCCGGCAA...,AAGACGGTCCGCACCCCCGGGGCCGCGGCGAACCTCGAGCTGATCT...,TCCCCGACGTCACCTTCGAGCTGCTGCGCGAGGGCGAGACGAAGGC...,1.740036,24.390222,0.071342,58858744,58858719,chr19,-,Internal_exon
2,A1BG,A1BG.3,3,11,0,CCGGGGTCTCTGGGGGGCTGGAGAAGCCTCCCTCATTCCTCCCAGG...,AGTGCCGGGGATTCTGGACTGGCTCCCTCCCCTCCTGTTGCAGCAC...,CCAGGGTGCTGTTGGTGTCCTCAGAAGTGCCGGGGATTCTGGACTG...,CACCATTTCTCTTTTCTCCCGTTGTCTAGAAAGCTGATGCAGCCGC...,11.529986,24.390222,0.472730,58858249,58858220,chr19,-,3_most_exon
3,A1BG,A1BG.4,4,11,4,GAAGATGCGTATCAGTATTATCAACCAATAGTTGATGAATTGTGTT...,ACCACTGCCGAGTTTCTGCCTGGAGCAGGGCCTCAAGGCCCTGGCA...,CCATCTGACATCTGATGCCTTCCCAACCACTGCCGAGTTTCTGCCT...,GATTCCTCTGTGTTCTTTCCCATTATCCCTCAGCTCTGAGTCCTCT...,1.386686,24.390222,0.056854,58857215,58857210,chr19,-,3_most_exon
4,A1BG,A1BG.5,5,11,4,AACGAATTTGCTAGTGATCTGGTTTACTGCCTTAGTAATATCTAGT...,GAAGATGCGTATCAGTATTATCAACCAATAGTTGATGAATTGTGTT...,CAGGGCCTCAAGGCCCTGGCACACAGAAGATGCGTATCAGTATTAT...,CCATCTGACATCTGATGCCTTCCCAACCACTGCCGAGTTTCTGCCT...,1.174863,24.390222,0.048169,58857165,58857142,chr19,-,3_most_exon


In [5]:

polyadb_df['start'] = polyadb_df['pas_pos'] - 150
polyadb_df['end'] = polyadb_df['pas_pos'] + 150

polyadb_bed = polyadb_df[['chrom', 'start', 'end', 'gene', 'gene_id', 'strand']].copy().reset_index(drop=True)

polyadb_bed.head()

polyadb_bed.to_csv("polyadb_coordinates.bed", sep='\t', header=False, index=False)


In [6]:
#Tissue types to compile data for

tissue_types = [
    'Adipose_Subcutaneous',
    'Adipose_Visceral_Omentum',
    'Adrenal_Gland',
    'Artery_Aorta',
    'Artery_Coronary',
    'Artery_Tibial',
    'Brain_Amygdala',
    'Brain_Anterior_cingulate_cortex_BA24',
    'Brain_Caudate_basal_ganglia',
    'Brain_Cerebellar_Hemisphere',
    ##'Brain_Cerebellum',
    'Brain_Cortex',
    'Brain_Frontal_Cortex_BA9',
    'Brain_Hippocampus',
    'Brain_Hypothalamus',
    'Brain_Nucleus_accumbens_basal_ganglia',
    'Brain_Putamen_basal_ganglia',
    'Brain_Spinal_cord_cervical_c-1',
    'Breast_Mammary_Tissue',
    'Cells_EBV-transformed_lymphocytes',
    'Cells_Transformed_fibroblasts',
    'Colon_Sigmoid',
    'Colon_Transverse',
    'Esophagus_Gastroesophageal_Junction',
    'Esophagus_Mucosa',
    'Esophagus_Muscularis',
    ##'Heart_Atrial_Appendage',
    'Heart_Left_Ventricle',
    'Liver',
    'Lung',
    'Muscle_Skeletal',
    'Nerve_Tibial',
    'Ovary',
    'Pancreas',
    'Pituitary',
    'Prostate',
    'Skin_Not_Sun_Exposed_Suprapubic',
    'Skin_Sun_Exposed_Lower_leg',
    'Small_Intestine_Terminal_Ileum',
    'Spleen',
    'Stomach',
    'Testis',
    'Thyroid',
    'Uterus',
    'Vagina',
    'Whole_Blood'
]


In [7]:

version_suffix = ''


In [8]:
#Compile apaQTL data from GTEx

def get_snp_pos(row) :
    if row['strand'] == '+' :
        return row['snp_pos'] - row['pas_pos'] + 175 - 1
    else :
        return row['pas_pos'] - row['snp_pos'] + 175

def get_snp_ref(row) :
    if row['strand'] == '+' :
        return row['snp_ref']
    else :
        if row['snp_ref'] == 'A' :
            return 'T'
        if row['snp_ref'] == 'C' :
            return 'G'
        if row['snp_ref'] == 'G' :
            return 'C'
        if row['snp_ref'] == 'T' :
            return 'A'

def get_snp_var(row) :
    if row['strand'] == '+' :
        return row['snp_var']
    else :
        if row['snp_var'] == 'A' :
            return 'T'
        if row['snp_var'] == 'C' :
            return 'G'
        if row['snp_var'] == 'G' :
            return 'C'
        if row['snp_var'] == 'T' :
            return 'A'

def get_var_seq(row) :
    ref_seq = row['wide_seq_ext']
    var_seq = ref_seq[:row['rel_snp_pos']] + row['rel_snp_var'] + ref_seq[row['rel_snp_pos']+1:]
    
    return var_seq

for tissue_type in tissue_types :
    
    print("Processing data for tissue = '" + str(tissue_type) + "'.")
    
    snp_df = pd.read_csv(tissue_type + '.cis.3aQTL.txt', sep='\t')

    snp_df['chr'] = snp_df['SNP'].apply(lambda x: x.split("_")[0])
    snp_df['start'] = snp_df['SNP'].apply(lambda x: x.split("_")[1])
    snp_df['end'] = snp_df['SNP'].apply(lambda x: str(int(x.split("_")[1]) + 1))
    snp_df['gene'] = snp_df['transcript'].apply(lambda x: x.split("|")[1])

    snp_df = snp_df[['chr', 'start', 'end', 'gene', 'SNP', 'transcript', 'beta', 't.stat', 'p.value']]

    snp_df.to_csv(tissue_type + version_suffix + ".cis.3aQTL.coordinates.bed", sep='\t', header=False, index=False)

    !bedtools intersect -a polyadb_coordinates.bed -b {tissue_type + version_suffix}.cis.3aQTL.coordinates.bed -wa -wb > polyadb_{tissue_type + version_suffix}_intersect.bed

    snp_bed_hg19 = pd.read_csv("polyadb_" + tissue_type + "_intersect.bed", sep='\t', error_bad_lines=False, names=['chrom', 'start', 'end', 'gene', 'gene_id', 'strand', 'chr_2', 'snp_pos', 'end_2', 'gene_2', 'snp_id', 'transcript_id', 'effect_size', 'test_statistic', 'p_val'])
    snp_bed_hg19 = snp_bed_hg19.query("gene == gene_2").copy().reset_index(drop=True)

    snp_bed_hg19 = snp_bed_hg19[['chrom', 'start', 'end', 'gene', 'gene_id', 'strand', 'chr_2', 'snp_pos', 'end_2', 'gene_2', 'snp_id', 'transcript_id', 'effect_size', 'test_statistic', 'p_val']]

    snp_bed_hg19['snp_ref'] = snp_bed_hg19['snp_id'].apply(lambda s: s.split("_")[2])
    snp_bed_hg19['snp_var'] = snp_bed_hg19['snp_id'].apply(lambda s: s.split("_")[3])
    
    polyadb_df_sel = polyadb_df[['wide_seq_ext', 'gene_id', 'pas', 'pas_pos', 'cut_mode', 'site_type', 'rpm', 'total_rpm', 'ratio']].copy().reset_index(drop=True)

    snp_df = snp_bed_hg19.join(polyadb_df_sel.set_index('gene_id'), on='gene_id').copy().reset_index(drop=True)
    
    snp_df['rel_snp_pos'] = snp_df.apply(get_snp_pos, axis=1)
    snp_df['rel_snp_ref'] = snp_df.apply(get_snp_ref, axis=1)
    snp_df['rel_snp_var'] = snp_df.apply(get_snp_var, axis=1)

    snp_df['wide_seq_ext_var'] = snp_df.apply(get_var_seq, axis=1)
    
    #Filter by lead SNPs (turned off)
    #lead_df = pd.read_csv("lead/" + tissue_type + ".lead.3aQTL.txt", sep='\t')
    #snp_df = snp_df.join(lead_df[['SNP']].set_index("SNP"), on='snp_id', how='inner')
    
    #Calculate mean PDUIs
    pdui_df = pd.read_csv(tissue_type + "_combined_All_PDUIs_clean.txt", sep='\t')
    pdui_df = pdui_df.join(snp_df[['transcript_id']].set_index('transcript_id'), on='event_id', how='inner').copy().reset_index(drop=True)

    gtex_samples = pdui_df.columns.values.tolist()[2:]

    def calc_mean_PDUI(row) :

        n_samples = 0.
        PDUI = 0.

        for gtex_sample in gtex_samples :
            if not np.isnan(row[gtex_sample]) :

                PDUI += row[gtex_sample]
                n_samples += 1.

        return PDUI / n_samples

    pdui_df['mean_PDUI'] = pdui_df.apply(calc_mean_PDUI, axis=1)

    snp_df = snp_df.join(pdui_df[['event_id', 'mean_PDUI']].set_index('event_id'), on='transcript_id', how='inner').copy().reset_index(drop=True)

    snp_df['transcript_id'] = snp_df['transcript_id'].apply(lambda x: x.split("|")[0])
    snp_df['snp_transcript_id'] = snp_df['snp_id'] + "_" + snp_df['transcript_id']
    
    transcript_end_dict = { row['transcript_id'] : -1 for _, row in snp_df.iterrows() }

    with open("/home/jlinder2/Downloads/GRCh37_latest_genomic.gtf", 'rt') as f :
        for line_raw in f.readlines() :

            a = line_raw.strip()

            if len(a.split("\t")) > 8 :
                start = a.strip().split("\t")[3]
                end = a.strip().split("\t")[4]

                strand = a.strip().split("\t")[6]

                t_id = ""
                if strand in ['+', '-'] :
                    t_id = a.strip().split("\t")[8].split(";")[1].split("\"")[1].split(".")[0]

                if t_id in transcript_end_dict :

                    if transcript_end_dict[t_id] == -1 :
                        if strand == '+' :
                            transcript_end_dict[t_id] = int(end)
                        else :
                            transcript_end_dict[t_id] = int(start)
                    else :
                        if strand == '+' :
                            transcript_end_dict[t_id] = max(transcript_end_dict[t_id], int(end))
                        else :
                            transcript_end_dict[t_id] = min(transcript_end_dict[t_id], int(start))

    qtl_cut_poses = []
    for _, row in snp_df.iterrows() :

        qtl_cut_poses.append(transcript_end_dict[row['transcript_id']])

    snp_df['qtl_cut_pos'] = qtl_cut_poses
    snp_df['snp_distance'] = np.abs(snp_df['qtl_cut_pos'] - snp_df['snp_pos'])

    #Drop duplicates
    snp_df['row_id'] = snp_df['gene_id'] + "_" + snp_df['snp_id'] + "_" + snp_df['transcript_id']
    snp_df = snp_df.sort_values(by='snp_id').drop_duplicates(subset=['row_id'], keep='first')

    #Save final apaQTL dataframe
    snp_df.to_csv("polyadb_" + tissue_type + version_suffix + "_SNPs.csv", sep='\t')


Processing data for tissue = 'Adipose_Subcutaneous'.
Processing data for tissue = 'Adipose_Visceral_Omentum'.
Processing data for tissue = 'Adrenal_Gland'.
Processing data for tissue = 'Artery_Aorta'.
Processing data for tissue = 'Artery_Coronary'.
Processing data for tissue = 'Artery_Tibial'.
Processing data for tissue = 'Breast_Mammary_Tissue'.
Processing data for tissue = 'Cells_Transformed_fibroblasts'.
Processing data for tissue = 'Colon_Sigmoid'.
Processing data for tissue = 'Colon_Transverse'.
Processing data for tissue = 'Esophagus_Gastroesophageal_Junction'.
Processing data for tissue = 'Esophagus_Mucosa'.
Processing data for tissue = 'Esophagus_Muscularis'.
Processing data for tissue = 'Nerve_Tibial'.
Processing data for tissue = 'Pancreas'.
Processing data for tissue = 'Pituitary'.
Processing data for tissue = 'Prostate'.
Processing data for tissue = 'Skin_Not_Sun_Exposed_Suprapubic'.
Processing data for tissue = 'Skin_Sun_Exposed_Lower_leg'.
Processing data for tissue = 'Sm

In [8]:
#Compute median effect sizes

exclude_tissues = [[]]
exclude_suffixes = ['']

#exclude_tissues = [[], ['Brain_Cortex', 'Brain_Frontal_Cortex_BA9'], ['Testis']]
#exclude_suffixes = ['', '_Exclude_Brain', '_Exclude_Testis']

for exclude_tissue, exclude_suffix in zip(exclude_tissues, exclude_suffixes) :

    print("Excluding " + str(exclude_tissue) + "...")
    
    snp_dict = {}

    for i, tissue_type in enumerate(tissue_types) :
        
        if tissue_type in exclude_tissue :
            continue

        print("Processing tissue = '" + tissue_type + "'")

        snp_df = pd.read_csv("polyadb_" + tissue_type + version_suffix + "_SNPs.csv", sep='\t')

        for _, row in snp_df.iterrows() :

            snp_id = row['snp_id'] + "__" + row['gene_id'] + "__" + row['transcript_id']

            effect_size = row['effect_size']
            p_val = row['p_val']

            if snp_id not in snp_dict :
                snp_dict[snp_id] = {
                    'effect_size' : [],
                    'p_val' : [],
                    'snp_id' : row['snp_id'],
                    'gene' : row['gene'],
                    'gene_id' : row['gene_id'],
                    'pas' : row['pas'],
                    'wide_seq_ext' : row['wide_seq_ext'],
                    'wide_seq_ext_var' : row['wide_seq_ext_var'],
                    'rel_snp_pos' : row['rel_snp_pos'],
                    'snp_pos' : row['rel_snp_pos'] - 105,
                    'mean_PDUI' : row['mean_PDUI'],
                    'site_type' : row['site_type'],
                }

            snp_dict[snp_id]['effect_size'].append(effect_size)
            snp_dict[snp_id]['p_val'].append(p_val)

    extended_snp_ids = []
    snp_ids = []
    genes = []
    gene_ids = []
    pases = []
    seqs = []
    seq_vars = []
    rel_snp_poses = []
    snp_poses = []
    effect_sizes = []
    p_vals = []
    n_tissues = []
    mean_PDUIs = []
    site_types = []

    for snp_id in snp_dict :

        extended_snp_ids.append(snp_id)
        snp_ids.append(snp_dict[snp_id]['snp_id'])
        effect_sizes.append(np.median(snp_dict[snp_id]['effect_size']))
        p_vals.append(np.min(snp_dict[snp_id]['p_val']))
        
        genes.append(snp_dict[snp_id]['gene'])
        gene_ids.append(snp_dict[snp_id]['gene_id'])
        pases.append(snp_dict[snp_id]['pas'])
        seqs.append(snp_dict[snp_id]['wide_seq_ext'])
        seq_vars.append(snp_dict[snp_id]['wide_seq_ext_var'])
        rel_snp_poses.append(snp_dict[snp_id]['rel_snp_pos'])
        snp_poses.append(snp_dict[snp_id]['snp_pos'])
        mean_PDUIs.append(snp_dict[snp_id]['mean_PDUI'])
        site_types.append(snp_dict[snp_id]['site_type'])

        n_tissues.append(len(snp_dict[snp_id]['effect_size']))

    median_df = pd.DataFrame({
        'extended_snp_id' : extended_snp_ids,
        'snp_id' : snp_ids,
        'gene' : genes,
        'gene_id' : gene_ids,
        'site_type' : site_types,
        'pas' : pases,
        'wide_seq_ext' : seqs,
        'wide_seq_ext_var' : seq_vars,
        'rel_snp_pos' : rel_snp_poses,
        'snp_pos' : snp_poses,
        'effect_size' : effect_sizes,
        'mean_PDUI' : mean_PDUIs,
        'p_val' : p_vals,
        'n_tissues' : n_tissues
    })
    median_df = median_df[['extended_snp_id', 'snp_id', 'gene', 'gene_id', 'site_type', 'pas', 'wide_seq_ext', 'wide_seq_ext_var', 'rel_snp_pos', 'snp_pos', 'effect_size', 'mean_PDUI', 'p_val', 'n_tissues']]

    median_df.to_csv("polyadb_median" + version_suffix + exclude_suffix + "_SNPs.csv", sep='\t')


Excluding []...
Processing tissue = 'Adipose_Subcutaneous'
Processing tissue = 'Adipose_Visceral_Omentum'
Processing tissue = 'Adrenal_Gland'
Processing tissue = 'Artery_Aorta'
Processing tissue = 'Artery_Coronary'
Processing tissue = 'Artery_Tibial'
Processing tissue = 'Brain_Amygdala'
Processing tissue = 'Brain_Anterior_cingulate_cortex_BA24'
Processing tissue = 'Brain_Caudate_basal_ganglia'
Processing tissue = 'Brain_Cerebellar_Hemisphere'
Processing tissue = 'Brain_Cortex'
Processing tissue = 'Brain_Frontal_Cortex_BA9'
Processing tissue = 'Brain_Hippocampus'
Processing tissue = 'Brain_Hypothalamus'
Processing tissue = 'Brain_Nucleus_accumbens_basal_ganglia'
Processing tissue = 'Brain_Putamen_basal_ganglia'
Processing tissue = 'Brain_Spinal_cord_cervical_c-1'
Processing tissue = 'Breast_Mammary_Tissue'
Processing tissue = 'Cells_EBV-transformed_lymphocytes'
Processing tissue = 'Cells_Transformed_fibroblasts'
Processing tissue = 'Colon_Sigmoid'
Processing tissue = 'Colon_Transverse'


In [9]:
median_df

,extended_snp_id,snp_id,gene,gene_id,site_type,pas,wide_seq_ext,wide_seq_ext_var,rel_snp_pos,snp_pos,effect_size,mean_PDUI,p_val,n_tissues
0,chr10_100176104_A_G__HPS1.13__NM_000195,chr10_100176104_A_G,HPS1,HPS1.13,3_most_exon,0,AATGAAAAGCGTGGATTGTGGAAAATACAGAAAAACATAAAGGAAA...,AATGAAAAGCGTGGATTGTGGAAAATACAGAAAAACATAAAGGAAA...,60,-45,-0.021652,0.141994,1.381527e-19,21
1,chr10_100176106_G_A__HPS1.13__NM_000195,chr10_100176106_G_A,HPS1,HPS1.13,3_most_exon,0,AATGAAAAGCGTGGATTGTGGAAAATACAGAAAAACATAAAGGAAA...,AATGAAAAGCGTGGATTGTGGAAAATACAGAAAAACATAAAGGAAA...,58,-47,-0.021652,0.141994,1.381527e-19,21
2,chr10_100176869_C_T__HPS1.12__NM_000195,chr10_100176869_C_T,HPS1,HPS1.12,3_most_exon,0,ACCCAGCAAGCTCTCTCCTACCCCAGGTAAAAGGTGCTCCTTTGCC...,ACCCAGCAAGCTCTCTCCTACCCCAGGTAAAAGGTGCTCCTTTGCC...,112,7,-0.022520,0.141994,5.248622e-24,28
3,chr10_100189138_A_G__HPS1.7__NM_182639,chr10_100189138_A_G,HPS1,HPS1.7,Intron,4,TCTTAGGCTCCAGTGGACCCCCGTGCCTCCTAAGGCTTGAGTGCAG...,TCTTAGGCTCCAGTGGACCCCCGTGCCTCCTAAGGCTTGAGTGCAG...,123,18,0.026503,0.216496,2.629988e-06,4
4,chr10_100189138_A_G__HPS1.8__NM_182639,chr10_100189138_A_G,HPS1,HPS1.8,Intron,0,AGGTACCCGTGTTCCTAGAGCACAGGACGCTGTCTGCGGCTCCCCA...,AGGTACCCGTGTTCCTAGAGCACAGGACGCTGTCTGCGGCTCCCCA...,79,-26,0.026503,0.216496,2.629988e-06,4
5,chr10_102746503_C_T__MRPL43.1__NM_032112,chr10_102746503_C_T,MRPL43,MRPL43.1,Intron,-1,GCAAGTCGGTCGAGGAGATCTCGACGCTGGTGCAGAAGCTGGCCGA...,GCAAGTCGGTCGAGGAGATCTCGACGCTGGTGCAGAAGCTGGCCGA...,196,91,-0.168484,0.669063,2.780783e-97,41
6,chr10_121435955_A_C__BAG3.2__NM_004281,chr10_121435955_A_C,BAG3,BAG3.2,Intron,0,GTTTCTTTTCATTGCCATTAAGAATACCATCTACAGAGAAATTCAG...,GTTTCTTTTCATTGCCATTAAGAATACCATCTACAGAGAAATTCAG...,271,166,-0.031654,0.840312,2.424760e-09,4
7,chr10_121589274_C_A__MCMBP.16__NM_024834,chr10_121589274_C_A,MCMBP,MCMBP.16,3_most_exon,0,ATAAAATGAAGGCTTCACTGGATTTGTGAGGATAAAATACATTTTC...,ATAAAATGAAGGCTTCACTGGATTTGTGAGGATAAAATACATTTTC...,231,126,0.029249,0.406829,3.402413e-06,4
8,chr10_121589274_C_A__MCMBP.15__NM_024834,chr10_121589274_C_A,MCMBP,MCMBP.15,3_most_exon,3,CTGGTTTTCATTTTTATAACTGGTAGACTAGACCTTCCTTAAACTT...,CTGGTTTTCATTTTTATAACTGGTAGACTAGACCTTCCTTAAACTT...,283,178,0.029249,0.406829,3.402413e-06,4
9,chr10_13160035_G_A__OPTN.2__NM_001008211,chr10_13160035_G_A,OPTN,OPTN.2,Intron,0,TCCGTTATGTTTCTAAGCCCTTGTATAAGGAAGAAAGTAAGTATTA...,TCCGTTATGTTTCTAAGCCCTTGTATAAGGAAGAAAGTAAGTATTA...,204,99,-0.038405,0.246714,1.228868e-15,22


In [10]:

len(median_df['snp_id'].unique())


5093

In [11]:
len(tissue_types)

44

In [12]:

lead_median_df = median_df.copy()

lead_median_df['cut_to_snp'] = np.abs(lead_median_df['snp_pos'] - 100)
lead_median_df = lead_median_df.sort_values(by='cut_to_snp', ascending=True).drop_duplicates(subset=['snp_id'], keep='first')

blacklist_ids = [
    'chr5_76728085_C_T'
]

lead_median_df = lead_median_df.loc[~lead_median_df['snp_id'].isin(blacklist_ids)]

len(lead_median_df['snp_id'].unique())


5092

In [17]:

lead_median_df = median_df.copy()

lead_median_df['cut_to_snp'] = np.abs(lead_median_df['snp_pos'] - 100)
lead_median_df = lead_median_df.sort_values(by='cut_to_snp', ascending=True).drop_duplicates(subset=['snp_id'], keep='first')

blacklist_ids = [
    'chr5_76728085_C_T'
]

lead_median_df = lead_median_df.loc[~lead_median_df['snp_id'].isin(blacklist_ids)]

len(lead_median_df.query("snp_pos >= 0 and snp_pos < 205")['snp_id'].unique())


3857

In [18]:

lead_median_df = median_df.copy()

lead_median_df['cut_to_snp'] = np.abs(lead_median_df['snp_pos'] - 100)
lead_median_df = lead_median_df.sort_values(by='cut_to_snp', ascending=True).drop_duplicates(subset=['snp_id'], keep='first')

blacklist_ids = [
    'chr5_76728085_C_T'
]

lead_median_df = lead_median_df.loc[~lead_median_df['snp_id'].isin(blacklist_ids)]

len(lead_median_df.query("snp_pos >= 0 and snp_pos < 146")['snp_id'].unique())


2940

In [19]:

lead_median_df = median_df.copy()

lead_median_df['cut_to_snp'] = np.abs(lead_median_df['snp_pos'] - 100)
lead_median_df = lead_median_df.sort_values(by='cut_to_snp', ascending=True).drop_duplicates(subset=['snp_id'], keep='first')

blacklist_ids = [
    'chr5_76728085_C_T'
]

lead_median_df = lead_median_df.loc[~lead_median_df['snp_id'].isin(blacklist_ids)]

len(lead_median_df.query("snp_pos >= 20 and snp_pos < 126")['snp_id'].unique())


2227